# Exercise 6

In [1]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
from nltk.metrics import jaccard_distance

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
!pip install -U spacy
import spacy
from scipy.stats import pearsonr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.1/29.1 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 28.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: blis
    Found existing installation: blis 0.7.11
    Uninstalling blis-0.7.11:
      Successfully uninstalled blis-0.7.11
  Attempting uninstall: thinc
    Found existing installation: thinc 8.2.5
    Uninstalling thinc-8.2.5:
      Successfully uninstalled thinc-8.2.5
  Attempting uninstall: spacy
    Found existing installation: spacy 3.7.5
    Uninstalling spacy-3.7.5:
      Successfully uninstalled spacy-3.7.5
ERROR: pip's d

## Importing the files

In [3]:
from google.colab import drive
import sys

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


**We decided not to use textserver because it had a limit for the queries and we couldn't use it for all the sentences**

In [6]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Exercises/STS.input.SMTeuroparl.txt',sep='\t',header=None)
#df = pd.read_csv('/content/drive/MyDrive/IHLT/Exercises/STS.input.SMTeuroparl.txt',sep='\t',header=None)

In [7]:
#We convert everything to lowercase
df[0]=df.apply(lambda x: x[0].lower(), axis = 1)
df[1]=df.apply(lambda x: x[1].lower(), axis = 1)

In [8]:
df['gs'] = pd.read_csv('/content/drive/MyDrive/Exercises/STS.gs.SMTeuroparl.txt',sep='\t',header=None)
#df['gs'] = pd.read_csv('/content/drive/MyDrive/IHLT/Exercises/STS.gs.SMTeuroparl.txt',sep='\t',header=None)

## Original results from exercise 2

We now present our best results from Exercise 2.

In [9]:
df['nltk_tokenized_0'] = df.apply(lambda x: [token for token in nltk.word_tokenize(x[0])], axis = 1)
df['nlkt_tokenized_1'] = df.apply(lambda x: [token for token in nltk.word_tokenize(x[1])], axis = 1)

In [10]:
df['jaccard_exercise_2'] = df.apply(lambda x: 1-jaccard_distance(set(x['nltk_tokenized_0']), set(x['nlkt_tokenized_1'])), axis = 1)

In [11]:
from scipy.stats import pearsonr
pearsonr(df['gs'], df['jaccard_exercise_2'])[0]

0.46249513975914963

## Original results from exercise 3

In [12]:
nlp = spacy.load("en_core_web_sm")

/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.7.1) was trained with spaCy v3.7.2 and may not be 100% compatible with the current version (3.8.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [13]:
df['lemmatized_filtered_0'] = df.apply(lambda x: [token.lemma for token in nlp(x[0]) if not token.is_stop], axis = 1)
df['lemmatized_filtered_1'] = df.apply(lambda x: [token.lemma for token in nlp(x[1]) if not token.is_stop], axis = 1)

In [14]:
df['jaccard_exercise_3'] = df.apply(lambda x: 1-jaccard_distance(set(x['lemmatized_filtered_0']), set(x['lemmatized_filtered_1'])), axis = 1)

In [15]:
pearsonr(df['gs'], df['jaccard_exercise_3'])[0]

0.49671900365312177

## WSD Analysis

We decided to tokenize the words from the sentences before calling the WSD algorithm.

### Directlly using all tokens

In [16]:
def get_wordnet_pos(category):
  '''Give the POS-tag used by wordnet from SpaCy's POS-tags.'''
  if category.startswith('ADJ'):
    return 'a'  # Adjective
  elif category.startswith('V'):
    return 'v'  # Verb
  elif category.startswith('N'):
    return 'n'  # Noun
  elif category.startswith('ADV'):
    return 'r'  # Adverb
  else:
    return None  # WordNet doesn't handle other POS tags

In [17]:
df['synsets_0'] = df.apply(lambda x: [nltk.wsd.lesk([token.text for token in nlp(x[0])], token.text, get_wordnet_pos(token.pos_)) for token in nlp(x[0])], axis = 1)

In [18]:
df['synsets_1'] = df.apply(lambda x: [nltk.wsd.lesk([token.text for token in nlp(x[1])], token.text, get_wordnet_pos(token.pos_)) for token in nlp(x[1])], axis = 1)

We decided to remove the None-s from the list of synsets for each pair of sentences, which turned out to perform better.

In [19]:
df['jaccard_synsets_naive'] = df.apply(lambda x: 1-jaccard_distance(set(x['synsets_0'])-{None}, set(x['synsets_1'])-{None}), axis = 1)
df['jaccard_synsets_naive_with_none'] = df.apply(lambda x: 1-jaccard_distance(set(x['synsets_0']), set(x['synsets_1'])), axis = 1)

In [20]:
print(pearsonr(df['gs'], df['jaccard_synsets_naive'])[0])
print(pearsonr(df['gs'], df['jaccard_synsets_naive_with_none'])[0])

0.4415824008205342
0.42986490022193113


As we expected, the results are not great at all, comparing naively the sets of the synsets is a super simple idea.

Later we will try a more sophisticated method for comparing the list of synsets.
A deeper problem is that the synsets provided by Lesk's algorithm are often wrong. This is especially the case in words that don't have an nltk POS-tag, because they are often given a synset that doesn't match the word category.

### Using only tokens with a pos tag in WorldNet

To solve the problem shown above, we remove those words completely which are not adjectives, verbs, nouns or adverbs (since these are the ones which can be used with WSD).

In [21]:
df['synsets_0_filtered'] = df.apply(lambda x: [nltk.wsd.lesk([token.text for token in nlp(x[0])], token.text, get_wordnet_pos(token.pos_)) for token in nlp(x[0]) if get_wordnet_pos(token.pos_)], axis = 1)

In [22]:
df['synsets_1_filtered'] = df.apply(lambda x: [nltk.wsd.lesk([token.text for token in nlp(x[1])], token.text, get_wordnet_pos(token.pos_)) for token in nlp(x[1]) if get_wordnet_pos(token.pos_)], axis = 1)

In [23]:
# We create a function to filter the lists with no elements, since the jaccard distance returns an error in that case
def jaccard_similarity_filtered(x):
  try:
    return 1-jaccard_distance(set(x['synsets_0_filtered']), set(x['synsets_1_filtered']))
  except:
    return 0

In [24]:
df['jaccard_synsets_filtered'] = df.apply(jaccard_similarity_filtered, axis = 1)
# if they are empty we save 0 as the simmilarity

In [25]:
pearsonr(df['gs'], df['jaccard_synsets_filtered'])[0]

0.35843957260481935

Let's get a sense of how many elements we lose on average due to lack of POS-tags.

In [28]:
# Non-filtered lists
mean0=0
mean1=0
for i in range(len(df['synsets_0'])):
  mean0+=len(df.loc[i,'synsets_0'])
  mean1+=len(df.loc[i,'synsets_1'])
print(f"The mean number of elements of the non-filtered lists is: {round(mean0/459,2)} and {round(mean1/459,2)}")

# Filtered lists
mean0=0
mean1=0
for i in range(len(df['synsets_0_filtered'])):
  mean0+=len(df.loc[i,'synsets_0_filtered'])
  mean1+=len(df.loc[i,'synsets_1_filtered'])
print(f"The mean number of elements of the filtered lists is: {round(mean0/459,2)} and {round(mean1/459,2)}")


The mean number of elements of the non-filtered lists is: 12.19 and 12.67
The mean number of elements of the filtered lists is: 5.52 and 5.38


* We get an unexpected outcome: the results got worse when we removed the synsets of words without a valid postag. We think that the explanation is that when you don't provide a postag, the WSD algorithm looks into every posible synset without taking into account the type of word. Unexpectedly, this blind choice makes our similarity score correlate more with the gold standard.

* There's a huge loss of synsets caused by the removal of words without POS-tags (we lose more than half). This could also account to the results being worse after removal.

* Up until this, when using the Jaccard similarity for two sets, we only checked if too synsets are exactly the same or not. To improve results we should make use of the fact that we can compare similarities between synsets as we did in the previous week. We will use Wu-Palmer similarity for this.

### Keeping the tokens for words without POS-tags

In this section we test that we don't perform the WSD algorithm for words which are not verbs, adverbs, nouns or adjectives but we also don't throw them away: we keep the tokens for the set. This way, the set for each sentence will contain tokens and synsets as well.

In [30]:
# We create a function to keep the tokens without postag
def synsets_token_coscious(x,i):
  return [
    nltk.wsd.lesk([token.text for token in nlp(x[0])], token.text, get_wordnet_pos(token.pos_)) if get_wordnet_pos(token.pos_)
    else token.text
    for token in nlp(x[i])
  ]

In [31]:
df['synsets_tokens_0'] = df.apply(synsets_token_coscious,i=0, axis = 1)

In [32]:
df['synsets_tokens_1'] = df.apply(synsets_token_coscious,i=1, axis = 1)

In [33]:
df['jaccard_synsets_tokens'] = df.apply(lambda x: 1-jaccard_distance(set(x['synsets_tokens_0']), set(x['synsets_tokens_1'])), axis = 1)
pearsonr(df['gs'], df['jaccard_synsets_tokens'])[0]

0.4807679545867132

We see that after doing this the result improves, but still far from the one obtained by lemmatizing.

### Wu-Palmer similarity computation

Our idea was to :
- iterate through the synsets from the first sentence,
- find the most similar synset from the second sentence for each synset in the first sencence,
- save this best similarity value for all of the synsets from the first sentence,
- and finally, average them across the first sentence.

This would be our similarity value for the two sentences.

In [34]:
### Using Wu-Palmer similarity insted of jaccard similarity
for i, synset_list in enumerate(df['synsets_0']):
  wu_mean=0

  for synset in set(synset_list)-{None}:
    maximum=0

    for synset2 in set(df['synsets_1'][i])-{None}:
      try:
        wup=synset.wup_similarity(synset2)
        if wup>maximum:
          maximum=wup
      except AttributeError:
        print('The synsets were none')
    wu_mean+=maximum
  df.loc[i,'Wu-Palmer Similarity']=wu_mean/len(synset_list)

In [35]:
pearsonr(df['gs'], df['Wu-Palmer Similarity'])[0]

0.28109582443789133

This result is far worse than we expected, we had high hopes for utilizing the Wu-Palmer similarity. In the end, the best results we have so far are from Week 3, using lemmatization and filtering the stopwords.